In [1]:
import math
import copy
class AddGaussianNoise():
    '''
    Add Gaussian Noise to bounding box
    '''

    def __init__(self,
                 mean=0.,
                 std=1.,
                 normalized: bool = True,
                 bernoulli_beta: float = 1.0):
        self.std = std
        self.ori_std=std
        self.mean = mean
        self.normalized = normalized
        # adding noise to every element by default
        self.bernoulli_beta = bernoulli_beta
        print('Noise: mean={0}, std={1}, beta={2}'.format(
            self.mean, self.std, self.bernoulli_beta))
        if self.ori_std >= 1.0:
            print("mix noise!")
    def __call__(self, data):
        # Gold Label
        if 'gold_bboxes' not in data.keys():
            data['gold_bboxes'] = copy.deepcopy(data['bboxes'])

        num_elemnts = data['bboxes'].size(0)
        beta = data['bboxes'].new_ones(num_elemnts) * self.bernoulli_beta
        element_with_noise = torch.bernoulli(beta).unsqueeze(dim=-1)

        if self.ori_std>=3.0:
            rand=torch.rand(1).item()
            if rand<=0.2:
                self.std=0.005
            elif rand<=0.4 and rand>0.2:
                self.std=0.01
            elif rand<=0.6 and rand>0.4:
                self.std=0.015
            elif rand<=0.8 and rand>0.6:
                self.std=0.02
            else:
                self.std=0.025


        elif self.ori_std>=2.0:
            rand=torch.rand(1).item()
            if rand<=0.25:
                self.std=0.005
            elif rand<=0.50 and rand>0.25:
                self.std=0.01
            elif rand<=0.75 and rand>0.5:
                self.std=0.015
            else:
                self.std=0.02

        elif self.ori_std >= 1.0: #mix noise
            rand=torch.rand(1).item()
            if rand<=0.33:
                self.std=0.005
            elif rand<=0.66 and rand>0.33:
                self.std=0.01
            else:
                self.std=0.02


        if self.normalized:
            data['bboxes'] = data['bboxes'] + torch.randn(
                data['bboxes'].size()) * self.std + self.mean
        else:
            canvas_width, canvas_height = data['canvas_size'][0], data[
                'canvas_size'][1]
            ele_x, ele_y = data['bboxes'][:, 0] * canvas_width, data[
                'bboxes'][:, 1] * canvas_height
            ele_w, ele_h = data['bboxes'][:, 2] * canvas_width, data[
                'bboxes'][:, 3] * canvas_height
            data['bboxes'] = torch.stack([ele_x, ele_y, ele_w, ele_h], dim=1)
            data['bboxes'] = data['bboxes'] + torch.randn(
                data['bboxes'].size()) * self.std + self.mean
            data['bboxes'][:, 0] /= canvas_width
            data['bboxes'][:, 1] /= canvas_height
            data['bboxes'][:, 2] /= canvas_width
            data['bboxes'][:, 3] /= canvas_height
        data['bboxes'][data['bboxes'] < 0] = 0.0
        data['bboxes'][data['bboxes'] > 1] = 1.0
        data['bboxes'] = data['bboxes'] * element_with_noise + data[
            'gold_bboxes'] * (1 - element_with_noise)
        return data

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1}, beta={2})'.format(
            self.mean, self.std, self.bernoulli_beta)

def convert_ltwh_to_ltrb(bbox):
    l, t, w, h = decapulate(bbox)
    r = l + w
    b = t + h
    return torch.stack([l, t, r, b], axis=-1)
class LabelDictSort():
    '''
    sort elements in one layout by their label
    '''
    def __init__(self, index2label=None):
        self.index2label = index2label

    def __call__(self, data):
        # NOTE: for refinement
        if 'gold_bboxes' not in data.keys():
            data['gold_bboxes'] = copy.deepcopy(data['bboxes'])

        labels = data['labels'].tolist()
        idx2label = [[idx, self.index2label[labels[idx]]] for idx in range(len(labels))]
        idx2label_sorted = sorted(idx2label, key=lambda x : x[1])
        idx_sorted = [d[0] for d in idx2label_sorted]
        data['bboxes'], data['labels'] = data['bboxes'][idx_sorted], data['labels'][idx_sorted]
        data['gold_bboxes'] = data['gold_bboxes'][idx_sorted]
        return data

class LexicographicSort():
    '''
    sort elements in one layout by their top and left postion
    '''

    def __call__(self, data):
        if 'gold_bboxes' not in data.keys():
            data['gold_bboxes'] = copy.deepcopy(data['bboxes'])
        l, t, _, _ = data['bboxes'].t()
        _zip = zip(*sorted(enumerate(zip(t, l)), key=lambda c: c[1:]))
        idx = list(list(_zip)[0])
        data['ori_bboxes'], data['ori_labels'] = data['gold_bboxes'], data[
            'labels']
        data['bboxes'], data['labels'] = data['bboxes'][idx], data['labels'][
            idx]
        data['gold_bboxes'] = data['gold_bboxes'][idx]
        return data

def decapulate(bbox):
    if len(bbox.size()) == 2:
        x1, y1, x2, y2 = bbox.T
    else:
        x1, y1, x2, y2 = bbox.permute(2, 0, 1)
    return x1, y1, x2, y2

class DiscretizeBoundingBox():

    def __init__(self, num_x_grid: int, num_y_grid: int) -> None:
        self.num_x_grid = num_x_grid
        self.num_y_grid = num_y_grid
        self.max_x = self.num_x_grid - 1
        self.max_y = self.num_y_grid - 1

    def discretize(self, bbox):
        """
        Args:
            continuous_bbox torch.Tensor: N * 4
        Returns:
            discrete_bbox torch.LongTensor: N * 4
        """
        cliped_boxes = torch.clip(bbox, min=0.0, max=1.0)
        x1, y1, x2, y2 = decapulate(cliped_boxes)
        discrete_x1 = torch.floor(x1 * self.max_x)
        discrete_y1 = torch.floor(y1 * self.max_y)
        discrete_x2 = torch.floor(x2 * self.max_x)
        discrete_y2 = torch.floor(y2 * self.max_y)
        return torch.stack(
            [discrete_x1, discrete_y1, discrete_x2, discrete_y2],
            dim=-1).long()

    def discretize_num(self, num: float) -> int:
        return int(math.floor(num * self.max_y))

    def __call__(self, data):
        discrete_bboxes = self.discretize(data['bboxes'])
        data['discrete_bboxes'] = discrete_bboxes
        return data



In [5]:
labels =['床(正视)', '壁挂空调(正视)', '台灯(正视)', '落地床头柜(正视)', '床(左视)', '落地床头柜(左视)', '横向装饰画(左视)', '扫地机器人(正视)', '落地灯(右视)', '横向装饰画(正视)', '装饰画(左视)', '床与床头柜组合(正视)', '装饰画(正视)', '吊灯(正视)', '竖向装饰画(正视)', '衣帽架(正视)', '吸顶灯(正视)', '落地梳妆台(正视)', '床(右视)', '落地床头柜(右视)', '四门衣柜(左视)', '装饰画(右视)', '空气净化器(正视)', '床尾凳(正视)', '横向装饰画(右视)', '落地灯(左视)', '吸尘器(正视)', '加湿器(正视)', '多门衣柜(右视)', '单人床尾凳(正视)', '落地空调(正视)', '落地梳妆台组合(正视)', '竖向装饰画(右视)', '四门衣柜(右视)', '壁挂空调(左视)', '落地灯(正视)', '空气净化器(右视)', '桌面加湿器(正视)', '多门衣柜(左视)', '落地无镜梳妆台组合(正视)', '扫地机器人(右视)', '双门衣柜(正视)', '竖向装饰画(左视)', '台灯(左视)', '壁挂空调(右视)', '吊灯(右视)', '洗地机(正视)', '落地梳妆台组合(右视)', '四门衣柜(正视)', '吸尘器(左视)', '落地梳妆台(右视)', '吸顶灯(左视)']

# labels = [
#     'Text', 'Image', 'Icon', 'List_Item', 'Text_Button', 'Toolbar',
#     'Web_View', 'Input', 'Card', 'Advertisement', 'Background_Image',
#     'Drawer', 'Radio_Button', 'Checkbox', 'Multi_Tab', 'Pager_Indicator',
#     'Modal', 'On_Off_Switch', 'Slider', 'Map_View', 'Button_Bar', 'Video',
#     'Bottom_Navigation', 'Number_Stepper', 'Date_Picker'
# ]
print(len(labels))

52


In [6]:
## test for reading pre-processed rico dataset
## convert preprocessed data to seq format (with discreting process embedded)

import torch
import io
import random # /home/shanxinyuan/common_disk/ygq/code/LayoutGeneration/LayoutDiffusion/data
ROOT_path = r"/home/ubuntu/ygq/"
with open(f'{ROOT_path}/LayOutDiffusion/data/raw_datasets/rico/pre_processed_20_25/train.pt', 'rb') as f:
    buffer = io.BytesIO(f.read())
    file=torch.load(buffer)

print(len(file))  ### 3279 
print(file[0:2])

270
[{'name': 'bedroom_708.json', 'labels': tensor([1, 2, 3, 4]), 'canvas_size': [800, 800], 'filtered': False, 'bboxes': tensor([[0.1515, 0.4286, 0.6861, 0.4251],
        [0.1361, 0.1131, 0.2987, 0.1040],
        [0.7781, 0.5064, 0.0774, 0.1374],
        [0.7478, 0.6229, 0.1603, 0.1659]])}, {'name': '1695285451064-anchorImage.json', 'labels': tensor([5, 6, 7, 8]), 'canvas_size': [800, 800], 'filtered': False, 'bboxes': tensor([[0.2512, 0.3787, 0.7125, 0.3875],
        [0.0999, 0.5246, 0.1639, 0.1692],
        [0.3335, 0.1547, 0.2782, 0.1135],
        [0.5839, 0.8173, 0.1474, 0.0929]])}]


/gpudisk2/liuluohang/anaconda3/envs/laydi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# 创建一个空的哈希表
vocab_size  = 186
hash_table = {"START": 0, "END": 1, "UNK": 2, "PAD": 3, "|": 4}

# 遍历labels列表，并将每个元素作为键添加到哈希表中
for i, label in enumerate(labels):
    hash_table[label] = i + 5 
for i in range(vocab_size-len(labels)-5 -1 ):
    hash_table[str(i)] = len(labels) + 5 + i
    
print(hash_table)

idx2label={}
for i in range(len(labels)): ###- 
    idx2label[i+1]=labels[i]
sort=LexicographicSort()
sort_type=LabelDictSort(index2label=idx2label)
discrete_fn=DiscretizeBoundingBox(128,128)


gaussian=AddGaussianNoise(mean=0,std=0.005,normalized=True,bernoulli_beta=1.0)

for i in range(len(file)):

    ### sort ###
    file[i]=sort_type(file[i])
    # file[i]=sort(file[i])

    ### noise for refine ###
    # file[i]=gaussian(file[i])

    ### ltrb ###
    file[i]['bboxes']=convert_ltwh_to_ltrb(file[i]['bboxes'])

    ### discretize ###
    file[i]=discrete_fn(file[i])

print(file[0])


all_layout=[]

for i,_ in enumerate(file):
    
    sample=file[i]
    layout=[]

   # layout.append('START')

    bbox=sample['discrete_bboxes']
    for idx,label in enumerate(sample['labels']):
        layout.append(labels[label-1])
        for pos in bbox[idx]:
            layout.append(str(pos.numpy()))
        if idx!=len(bbox)-1:
            layout.append("|")
    
    # layout.append('END ]')

    str_layout=' '.join(layout)
    all_layout.append(str_layout)
print(all_layout[0])
print(all_layout[1])


data=open(f"{ROOT_path}/LayOutDiffusion/data/processed_datasets/RICO_ltrb_lex/src1_train.txt",'w+')
for _,layout in enumerate(all_layout): 
    print(layout,file=data)
data.close()


{'START': 0, 'END': 1, 'UNK': 2, 'PAD': 3, '|': 4, '床(正视)': 5, '壁挂空调(正视)': 6, '台灯(正视)': 7, '落地床头柜(正视)': 8, '床(左视)': 9, '落地床头柜(左视)': 10, '横向装饰画(左视)': 11, '扫地机器人(正视)': 12, '落地灯(右视)': 13, '横向装饰画(正视)': 14, '装饰画(左视)': 15, '床与床头柜组合(正视)': 16, '装饰画(正视)': 17, '吊灯(正视)': 18, '竖向装饰画(正视)': 19, '衣帽架(正视)': 20, '吸顶灯(正视)': 21, '落地梳妆台(正视)': 22, '床(右视)': 23, '落地床头柜(右视)': 24, '四门衣柜(左视)': 25, '装饰画(右视)': 26, '空气净化器(正视)': 27, '床尾凳(正视)': 28, '横向装饰画(右视)': 29, '落地灯(左视)': 30, '吸尘器(正视)': 31, '加湿器(正视)': 32, '多门衣柜(右视)': 33, '单人床尾凳(正视)': 34, '落地空调(正视)': 35, '落地梳妆台组合(正视)': 36, '竖向装饰画(右视)': 37, '四门衣柜(右视)': 38, '壁挂空调(左视)': 39, '落地灯(正视)': 40, '空气净化器(右视)': 41, '桌面加湿器(正视)': 42, '多门衣柜(左视)': 43, '落地无镜梳妆台组合(正视)': 44, '扫地机器人(右视)': 45, '双门衣柜(正视)': 46, '竖向装饰画(左视)': 47, '台灯(左视)': 48, '壁挂空调(右视)': 49, '吊灯(右视)': 50, '洗地机(正视)': 51, '落地梳妆台组合(右视)': 52, '四门衣柜(正视)': 53, '吸尘器(左视)': 54, '落地梳妆台(右视)': 55, '吸顶灯(左视)': 56, '0': 57, '1': 58, '2': 59, '3': 60, '4': 61, '5': 62, '6': 63, '7': 64, '8': 65, '9': 66, '10': 67, '11': 68, '12': 69, '1

Noise: mean=0, std=0.005, beta=1.0
{'name': 'bedroom_708.json', 'labels': tensor([3, 2, 1, 4]), 'canvas_size': [800, 800], 'filtered': False, 'bboxes': tensor([[0.7781, 0.5064, 0.8554, 0.6438],
        [0.1361, 0.1131, 0.4349, 0.2172],
        [0.1515, 0.4286, 0.8376, 0.8536],
        [0.7478, 0.6229, 0.9082, 0.7887]]), 'gold_bboxes': tensor([[0.7781, 0.5064, 0.0774, 0.1374],
        [0.1361, 0.1131, 0.2987, 0.1040],
        [0.1515, 0.4286, 0.6861, 0.4251],
        [0.7478, 0.6229, 0.1603, 0.1659]]), 'discrete_bboxes': tensor([[ 98,  64, 108,  81],
        [ 17,  14,  55,  27],
        [ 19,  54, 106, 108],
        [ 94,  79, 115, 100]])}


台灯(正视) 98 64 108 81 | 壁挂空调(正视) 17 14 55 27 | 床(正视) 19 54 106 108 | 落地床头柜(正视) 94 79 115 100
床(左视) 31 48 122 97 | 扫地机器人(正视) 74 103 92 115 | 横向装饰画(左视) 42 19 77 34 | 落地床头柜(左视) 12 66 33 88


In [190]:
# ! ls /home/shanxinyuan/common_disk/ygq/code/LayoutGeneration/LayoutDiffusion/data/processed_datasets/my_ltrb_lex_edit

ls: cannot access '/home/shanxinyuan/common_disk/ygq/code/LayoutGeneration/LayoutDiffusion/data/processed_datasets/my_ltrb_lex_edit': No such file or directory


In [3]:
! cd /home/shanxinyuan/common_disk/code/PTM/jiabu/0224_跑批/tools/img_test/json_files_202406/ && ls

0611			      6月新增客厅SKU生产.xlsx
202404.syn.config	      color_data_202406.json
202406_skus_angle_info.json   get_sku_base_info.py
202406_skus_brand_info.json   get_sku_toudi_info.py
202406_skus_cate_info.json    jiabu_skus_valid_0607.json
202406_skus_shopId_info.json  main_skus.txt
202406_skus_style_info.json   select_main_sku.py
202406_skus_toudi_info.txt    sku_base_info_0611.json
202406_skus_url_info.json


In [6]:
! cd /home/shanxinyuan/common_disk/ygq/code/2D-Product-matching/data/ && python step0_sku_info_list_preprocess_202406.py

{'扫地机器人', '茶几右L电视柜组合', '竖向装饰画', '落地灯', '方形茶几', '右L型电视柜', '无腿电视柜', '异形组合茶几', '左L型电视柜', '吸尘器', '无腿茶几', '茶几', '茶吧机', '吊灯', '吸顶灯', '落地空调', '茶几左L电视柜组合', '横向装饰画', '壁挂空调', '电视', '带壳扫地机器人', '空气净化器', '茶几电视柜组合', '三人沙发', '单人沙发', '支架电视', '洗地机', '双人沙发', '装饰画', '方形无腿茶几', '左L型沙发', '方形落地空调', '电视柜', '立体边几', '右L型沙发', '边几'}
{'茶几右L电视柜组合', '竖向装饰画', '落地灯', '方形茶几', '右L型电视柜', '无腿电视柜', '异形组合茶几', '左L型电视柜', '无腿茶几', '茶几', '茶吧机', '吊灯', '吸顶灯', '茶几左L电视柜组合', '横向装饰画', '茶几电视柜组合', '三人沙发', '单人沙发', '支架电视', '双人沙发', '装饰画', '方形无腿茶几', '左L型沙发', '电视柜', '立体边几', '右L型沙发', '边几'}


In [8]:
# ! cd /home/shanxinyuan/common_disk/ygq/code/2D-Product-matching/ && python run_step_0_all_rule_202406.py 

set()
2008 691 74
balcony all count: 0
bedroom all count: 229
livingroom all count: 0
washingroom all count: 0


In [103]:
! cd /home/shanxinyuan/common_disk/ygq/code/2D-Product-matching/ && python run_step_0_all_rule_202406.py 


set()
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------


In [96]:
! cd /home/shanxinyuan/common_disk/ygq/code/2D-Product-matching/ && python run_step_0_all_rule_202406.py  
## 里面都是 文字键 
## bedroom 没有经过这个分支


set()
[]
balcony all count: 0
[]
bedroom all count: 0
[['电视(正视)', '茶几(正视)', '落地灯(左视)'], ['三人沙发(左视)', '吊灯(左视)', '吸尘器(左视)', '茶几(左视)', '落地灯(正视)'], ['无腿电视柜(正视)', '电视(正视)', '茶几(正视)', '落地灯(正视)'], ['方形无腿茶几(正视)', '电视(正视)', '电视柜(正视)'], ['三人沙发(正视)', '壁挂空调(正视)', '茶几(正视)', '落地灯(右视)', '装饰画(正视)'], ['三人沙发(正视)', '扫地机器人(正视)', '横向装饰画(正视)', '茶几(正视)', '落地灯(正视)'], ['电视(右视)', '电视柜(右视)', '茶几(右视)', '落地空调(右视)'], ['三人沙发(正视)', '吊灯(正视)', '扫地机器人(正视)', '横向装饰画(正视)', '茶几(正视)'], ['吊灯(正视)', '左L型沙发(正视)', '茶几(正视)', '落地灯(右视)', '落地空调(左视)'], ['三人沙发(正视)', '吊灯(正视)', '方形落地空调(正视)', '茶几(正视)', '装饰画(正视)'], ['三人沙发(正视)', '吊灯(正视)', '横向装饰画(正视)', '茶几(正视)', '落地空调(右视)'], ['单人沙发(右视)', '电视柜(正视)', '落地空调(正视)', '边几(正视)'], ['三人沙发(正视)', '横向装饰画(正视)', '茶几(正视)', '落地灯(左视)', '落地空调(正视)'], ['电视(正视)', '电视柜(正视)', '落地灯(左视)'], ['三人沙发(正视)', '吊灯(正视)', '无腿茶几(正视)', '茶吧机(正视)'], ['三人沙发(正视)', '方形落地空调(正视)', '横向装饰画(正视)', '茶几(正视)'], ['电视(正视)', '电视柜(正视)', '空气净化器(正视)'], ['三人沙发(正视)', '壁挂空调(正视)', '竖向装饰画(正视)'], ['电视(正视)', '电视柜(正视)', '落地空调(右视)'], ['右L型沙发(正视)', '茶几(正视)', 

In [110]:
! cd /home/shanxinyuan/common_disk/ygq/code/2D-Product-matching/data && python step0_sku_info_list_preprocess_202406.py

{'三人沙发', '右L型沙发', '边几', '无腿电视柜', '方形茶几', '方形无腿茶几', '右L型电视柜', '洗地机', '扫地机器人', '吊灯', '异形组合茶几', '装饰画', '横向装饰画', '电视', '茶几', '竖向装饰画', '茶几右L电视柜组合', '落地灯', '壁挂空调', '无腿茶几', '立体边几', '支架电视', '方形落地空调', '吸尘器', '茶几电视柜组合', '空气净化器', '落地空调', '电视柜', '带壳扫地机器人', '双人沙发', '左L型电视柜', '吸顶灯', '左L型沙发', '茶吧机', '单人沙发', '茶几左L电视柜组合'}
{'三人沙发', '右L型沙发', '边几', '无腿电视柜', '方形茶几', '方形无腿茶几', '右L型电视柜', '吊灯', '异形组合茶几', '装饰画', '横向装饰画', '茶几', '竖向装饰画', '茶几右L电视柜组合', '落地灯', '无腿茶几', '立体边几', '支架电视', '茶几电视柜组合', '电视柜', '双人沙发', '左L型电视柜', '吸顶灯', '左L型沙发', '茶吧机', '单人沙发', '茶几左L电视柜组合'}


In [112]:
! cd /home/shanxinyuan/common_disk/ygq/code/2D-Product-matching/ && python run_step_0_all_rule_202406.py 


set()


In [107]:
! cd /home/shanxinyuan/common_disk/ygq/code/2D-Product-matching/ && python run_step_1_matching_202406.py

  0%|                                                     | 0/8 [00:00<?, ?it/s]processing: balcony_mini_group_set_list.json
processing: balcony_normal_set_list.json
processing: bedroom_mini_group_set_list.json
processing: bedroom_normal_set_list.json
processing: livingroom_mini_group_set_list.json

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 13.47it/s]

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 13.62it/s]

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 208.63it/s]

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 227.50it/s]

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 111.75it/s]

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 33.76it/s]

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  9.38it/s]

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  9.60it/s]

100%|██████████████████████████████████████

In [98]:
! cd /home/shanxinyuan/common_disk/ygq/code/2D-Product-matching/ && python run_step_1_matching_202406.py

  0%|                                                     | 0/8 [00:00<?, ?it/s]processing: balcony_mini_group_set_list.json
processing: balcony_normal_set_list.json
processing: bedroom_mini_group_set_list.json
processing: bedroom_normal_set_list.json
processing: livingroom_mini_group_set_list.json

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 104.04it/s]

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 93.51it/s]

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 44.87it/s]

100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 2434.30it/s]

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 39.02it/s]

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 34.61it/s]

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 424.95it/s]

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 186.93it/s]

100%|██████████████████████████████████████

In [13]:
import json
# 55523 
with open (r"/home/shanxinyuan/common_disk/code/PTM/jiabu/0224_跑批/tools/img_test/json_files_202406/202406_skus_angle_info.json" ,'r',encoding = "utf-8") as file:
    data = json.load(file)
    print("202406_skus_angle_info:",len(data)) ###- 
with open (r"/home/shanxinyuan/common_disk/code/PTM/jiabu/0224_跑批/tools/img_test/json_files_202406/202406_skus_brand_info.json" ,'r',encoding = "utf-8") as file:
    data = json.load(file)
    print("202406_skus_brand_info.json:",len(data)) ###- 
with open (r"/home/shanxinyuan/common_disk/code/PTM/jiabu/0224_跑批/tools/img_test/json_files_202406/202406_skus_cate_info.json" ,'r',encoding = "utf-8") as file:
    data = json.load(file)
    print("202406_skus_cate_info.json:",len(data)) ###- 
with open (r"/home/shanxinyuan/common_disk/code/PTM/jiabu/0224_跑批/tools/img_test/json_files_202406/202406_skus_shopId_info.json" ,'r',encoding = "utf-8") as file:
    data = json.load(file)
    print("202406_skus_shopId_info.json:",len(data)) ###- 
with open (r"/home/shanxinyuan/common_disk/code/PTM/jiabu/0224_跑批/tools/img_test/json_files_202406/color_data_202406.json" ,'r',encoding = "utf-8") as file:
    data = json.load(file)
    print("color_data_202406.json:",len(data)) ###- 

with open (r"/home/shanxinyuan/common_disk/code/PTM/jiabu/0224_跑批/tools/img_test/json_files_202406/jiabu_skus_valid_0607.json" ,'r',encoding = "utf-8") as file:
    data = json.load(file)
    print("jiabu_skus_valid_0607.json:",len(data)) ###- 

202406_skus_angle_info: 55522
202406_skus_brand_info.json: 55523
202406_skus_cate_info.json: 55523
202406_skus_shopId_info.json: 55523
color_data_202406.json: 55479
jiabu_skus_valid_0607.json: 118898


In [14]:
with open (r"/home/shanxinyuan/common_disk/ygq/code/2D-Product-matching/config/202406/main_sku_info_list_202406.json" ,'r',encoding = "utf-8") as file:
    data = json.load(file)
    print("jiabu_skus_valid_0607.json:",len(data)) ###- 

jiabu_skus_valid_0607.json: 52265
